In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import dm4bem

# Defining the simulation parameters

## Define the setup

In [23]:
controller = True

imposed_time_step = False
delta_t = 500    # s, imposed time step

calculate_step_response = False

output_nodes = [25]

## Outside conditions

In [24]:
To_ss = 10          # °C
To_stp = 10         # °C

# weather data
filename = './weather_data/CHE_AG_Wuerenlingen.066470_TMYx.2009-2023.epw'
[data, meta] = dm4bem.read_epw(filename, coerce_year=None)
weather_data = data[["temp_air", "dir_n_rad", "dif_h_rad"]]
weather_data = weather_data.sort_index()



first timestamp: 2015-03-01 00:00:00+01:00
last timestamp: 2022-04-30 23:00:00+01:00
temp_air     15.5
dir_n_rad     0.0
dif_h_rad     0.0
Name: 2020-08-31 23:00:00+01:00, dtype: float64


## Controller

In [25]:
Tisp = 22     # °C

if controller:
    Kp = 1e3
else:
    Kp = 0

## Ventilation

In [26]:
ACH = 1

## Material properties

In [27]:
# concrete
lam_concrete = 1.4                          # W/(m⋅K)
c_concrete = 880.0                          # J/(kg⋅K)
rho_concrete = 2300.0                       # kg/m3

# insulation
lam_insulation = 0.004                      # W/(m⋅K)
c_insulation = 1210.0                       # J/(kg⋅K)
rho_insulation = 55.0                       # kg/m3

# ceiling
lam_ceiling = 0.5                           # W/(m⋅K)
c_ceiling = 1000.0                          # J/(kg⋅K)
rho_ceiling = 200.0                         # kg/m3

# windows
u_window = 1.2                              # W/(m²⋅K)

# air
cp_air = 1006                               # J/(kg⋅K)
rho_air = 1.2                               # kg/m3

## Convection coefficients

In [28]:
h_in = 7.7                                  # W/(m²⋅K)
h_out = 25                                  # W/(m²⋅K)

h_c_in = 7.7                                # W/(m²⋅K)
h_c_out = 25                                # W/(m²⋅K)

## Geometry

In [29]:
height_room = 2.5                             # m

w_concrete = 0.4                            # m
w_insulation = 0.05                         # m
w_ceiling = 0.8                             # m

width_wall1 = 5.3                             # m
width_wall2 = 9.5                            # m
width_wall3 = 9.2                            # m
width_wall4 = 3.3                             # m

width_windows = 2.5                           # m

A_window_1 = width_windows*height_room      # m²
A_wall_1 = width_wall1*height_room-A_window_1       # m²

A_window_2 = width_windows*height_room * 2  # m²
A_wall_2 = width_wall2*height_room-A_window_2      # m²

A_ceiling = width_wall1*width_wall3 - width_wall3*(width_wall1-width_wall4)/2   # m²

angle_wall = 80                             # °

# Orientation definition
wall_1_orientation = {'slope': 90,
                      'azimuth': -90,
                      'latitude': 45}
albedo_wall_1 = 0.55

wall_2_orientation = {'slope': 90,
                      'azimuth': 90-angle_wall,
                      'latitude': 45}
albedo_wall_2 = 0.55

ceiling_orientation = {'slope': 180,
                       'azimuth': 0,
                       'latitude':45}
albedo_ceiling = 0.55

## Emmisivities & view factors

In [30]:
sigma = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant

eps_i = 0.96
eps_a_w = 0.8
eps_a_c = 0.7
# check calculation ------------------------------------------------------------------------------------
A12 = width_wall1/width_wall2
F12 = (A12+1-np.sqrt(A12**2+1-2*A12*np.cos((angle_wall/360) * 2*np.pi)))/2
H1c = width_wall2/width_wall1
F1c = 0.5*(1+H1c - np.sqrt(1+H1c**2))
H2c = width_wall1/width_wall2
F2c = 0.5*(1+H2c - np.sqrt(1+H2c**2))

# Defining the matrices and vectrors

## A

In [31]:
n_nodes = 25
n_flows = 39

A = np.zeros((n_flows+1, n_nodes+1))
A[0,0] = 1
A[1,0] = -1; A[1,1] = 1
A[2,1] = -1; A[2,2] = 1
A[3,2] = -1; A[3,3] = 1
A[4,3] = -1; A[4,4] = 1
A[5,4] = -1; A[5,5] = 1
A[6,5] = -1; A[6,6] = 1
A[7,6] = -1; A[7,7] = 1
A[8,7] = -1; A[8,8] = 1
A[9,8] = -1; A[9,25] = 1
A[10,8] = -1; A[10,18] = 1
A[11,9] = 1
A[12,9] = -1; A[12,25] = 1
A[13,8] = -1; A[13,19] = 1
A[14,9] = -1; A[14,18] = 1
A[15,10] = 1
A[16,10] = -1; A[16,11] = 1
A[17,11] = -1; A[17,12] = 1
A[18,12] = -1; A[18,13] = 1
A[19,13] = -1; A[19,14] = 1
A[20,14] = -1; A[20,15] = 1
A[21,15] = -1; A[21,16] = 1
A[22,16] = -1; A[22,17] = 1
A[23,17] = -1; A[23,18] = 1
A[24,18] = -1; A[24,25] = 1
A[25,9] = -1; A[25,19] = 1
A[26,19] = 1
A[27,19] = -1; A[27,25] = 1
A[28,20] = 1
A[29,20] = -1; A[29,21] = 1
A[30,21] = -1; A[30,22] = 1
A[31,22] = -1; A[31,23] = 1
A[32,23] = -1; A[32,24] = 1
A[33,24] = -1; A[33,8] = 1
A[34,24] = -1; A[34,9] = 1
A[35,24] = -1; A[35,18] = 1
A[36,24] = -1; A[36,19] = 1
A[37,24] = -1; A[37,25] = 1
A[38,25] = 1
A[39,25] = 1


## G

In [32]:
G = np.zeros(n_flows+1)

# wall 1
G[0] = h_out*A_wall_1

# conduction concrete
for i in [5,6,7,8]:
    G[i] = lam_concrete*A_wall_1/w_concrete

# conduction insulation
for i in [1,2,3,4]:
    G[i] = lam_insulation*A_wall_1/w_insulation

G[9] = h_in*A_wall_1

# window 1
G[11] = u_window*A_window_1
G[12] = h_in*A_window_1

# wall 2
G[15] = h_out*A_wall_2

# conduction concrete
for i in [20,21,22,23]:
    G[i] = lam_concrete*A_wall_2/w_concrete

# conduction insulation
for i in [16,17,18,19]:
    G[i] = lam_insulation*A_wall_2/w_insulation

G[24] = h_in*A_wall_2

# window 2
G[26] = u_window*A_window_2
G[27] = h_in*A_window_2

# ceiling
G[28] = h_c_out*A_ceiling

for i in [29,30,31,32]:
    G[i] = lam_ceiling*A_ceiling/w_ceiling

G[37] = h_c_in*A_ceiling

# Ventilation
V_a = A_ceiling*height_room
V_dot_a = ACH*V_a/3600
G[38] = rho_air*V_dot_a*cp_air

# Controller
G[39] = Kp

# View Factors
#-------------------------------------------------------


## C

In [33]:
C = np.zeros(n_nodes+1)

# wall 1
for i in [5,6,7]:
    C[i] = c_concrete*rho_concrete*w_concrete*A_wall_1 / 3

for i in [1,2,3]:
    C[i] = c_insulation*rho_insulation*w_insulation*A_wall_1 / 3

# wall 2
for i in [15,16,17]:
    C[i] = c_concrete*rho_concrete*w_concrete*A_wall_2 / 3

for i in [11,12,13]:
    C[i] = c_insulation*rho_insulation*w_insulation*A_wall_2 / 3

# ceiling
for i in [21,22,23]:
    C[i] = c_ceiling*rho_ceiling*w_ceiling*A_ceiling / 3

C[25] = cp_air*rho_air*V_a

## b for steady state

In [34]:
b_ss = np.zeros(n_flows+1)

for i in [0,11,15,26,28,38]:
    b_ss[i] = To_ss

b_ss[39] = Tisp


## f for steady state

In [35]:
f_ss = np.zeros(n_nodes+1)


## y

In [36]:
y = np.zeros(n_nodes+1)

for i in output_nodes:
    y[i] = 1

# Solving the steady state

In [37]:
diag_G = np.diag(G)

theta_ss = np.linalg.inv(A.T @ diag_G @ A) @ (A.T @ diag_G @ b_ss + f_ss)
print(f'Temperature inside: {np.around(theta_ss[25], 2)} °C')

Temperature inside: 21.33 °C


# Dynamic simulation

## b and f sources for a dynamic simulation

In [38]:
b = np.zeros(n_flows+1)
f = np.zeros(n_nodes+1)

## Assemble the thermal circuit

In [39]:
node_labels = [f"θ{i}" for i in range(n_nodes+1)]
flow_labels = [f"q{i}" for i in range(n_flows+1)]

A = pd.DataFrame(A, index=flow_labels, columns=node_labels)
G = pd.Series(G, index=flow_labels, name="G")
C = pd.Series(C, index=node_labels, name="C")
b = pd.Series(b, index=flow_labels, name="b").astype("object")
f = pd.Series(f, index=node_labels, name="f").astype("object")
y = pd.Series(y, index=node_labels, name="y")

for i in [0,11,15,26,28,38]:
    b[f"q{i}"] = "To"
b["q39"] = "Tisp"

f["θ0"] = "phi_1_o"; f["θ8"] = "phi_1_i"
f["θ10"] = "phi_2_o"; f["θ18"] = "phi_2_i"
f["θ20"] = "phi_c_o"; f["θ24"] = "phi_c_i"
f["θ25"] = "Q_dot"

TC = {
    "A": A,
    "G": G,
    "C": C,
    "b": b,
    "f": f,
    "y": y,
}

[As, Bs, Cs, Ds, us] = dm4bem.tc2ss(TC)

## Eigenvalues analysis

In [40]:
lam_eigen = np.linalg.eig(As)[0]

# time step
delta_tmax = 2 * min(-1. / lam_eigen) * 0.5    # reducing the max time step by 0.5 for stability reasons
dm4bem.print_rounded_time('Δtmax', delta_tmax)

if imposed_time_step:
    dt = delta_t
else:
    dt = dm4bem.round_time(delta_tmax)
dm4bem.print_rounded_time('dt', dt)

# settling time
t_settle = 4 * max(-1 / lam_eigen)
dm4bem.print_rounded_time('t_settle', t_settle)

# duration: next multiple of 3600 s that is larger than t_settle
duration = np.ceil(t_settle / 3600) * 3600
dm4bem.print_rounded_time('duration', duration)

Δtmax = 106 s = 1.8 min
dt = 60 s
t_settle = 1970903 s = 547.5 h
duration = 1972800 s = 548.0 h


## Step response

In [41]:
if calculate_step_response:

    # Create input_data_set

    n_stp = int(np.floor(duration / dt))    # number of time steps
    time_stp = pd.date_range(start="2000-01-01 00:00:00",
                               periods=n_stp, freq=f"{int(dt)}s")

    print(f'Number of time steps: {n_stp}')

    To_stp = To_stp * np.ones(n_stp)
    Ti_sp_stp = Tisp * np.ones(n_stp)
    phi_1_o_stp = 0 * np.ones(n_stp)
    Q_dot_stp = phi_1_i_stp = phi_2_o_stp = phi_2_i_stp = phi_c_o_stp = phi_c_i_stp = phi_1_o_stp

    data_stp = {'To': To_stp, 'Tisp': Ti_sp_stp, 'phi_1_o': phi_1_o_stp, 'phi_1_i': phi_1_i_stp,
                'phi_2_o': phi_2_o_stp, 'phi_2_i': phi_2_i_stp, 'phi_c_o': phi_c_o_stp,
                'phi_c_i': phi_c_i_stp, 'Q_dot': Q_dot_stp}
    input_data_set_stp = pd.DataFrame(data_stp, index=time_stp)
    u_stp = dm4bem.inputs_in_time(us, input_data_set_stp)


    # Initial conditions
    theta_exp_stp = pd.DataFrame(index=u_stp.index)     # empty df with index for explicit Euler
    theta_imp_stp = pd.DataFrame(index=u_stp.index)     # empty df with index for implicit Euler

    theta_0 = 0.0                    # initial temperatures
    theta_exp_stp[As.columns] = theta_0      # fill θ for Euler explicit with initial values θ0
    theta_imp_stp[As.columns] = theta_0      # fill θ for Euler implicit with initial values θ0

    I = np.eye(As.shape[0])     # identity matrix
    for k in range(u_stp.shape[0] - 1):
        theta_exp_stp.iloc[k + 1] = (I + dt * As)\
            @ theta_exp_stp.iloc[k] + dt * Bs @ u_stp.iloc[k]
        theta_imp_stp.iloc[k + 1] = np.linalg.inv(I - dt * As)\
            @ (theta_imp_stp.iloc[k] + dt * Bs @ u_stp.iloc[k])

    # outputs
    y_exp_stp = (Cs @ theta_exp_stp.T + Ds @  u_stp.T).T
    y_imp_stp = (Cs @ theta_imp_stp.T + Ds @  u_stp.T).T

### Plot the data

In [42]:
if calculate_step_response:
    # plot results
    y_stp = pd.concat([y_exp_stp, y_imp_stp], axis=1, keys=['Explicit', 'Implicit'])
    # Flatten the two-level column labels into a single level
    y_stp.columns = y_stp.columns.get_level_values(0)

    ax = y_stp.plot()
    ax.set_xlabel('Time')
    ax.set_ylabel('Indoor temperature, $\\theta_i$ / °C')
    ax.set_title(f'Time step: $dt$ = {dt:.0f} s; $dt_{{max}}$ = {delta_tmax:.0f} s')
    #ax.set_xlim(["2000-01-01 00:00:00", "2000-01-02 00:00:00"])
    plt.show()

## Dynamic simulation with weather data